In [9]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [58]:
import json
import gzip
import pandas as pd
import glob
import tempfile
from importlib import reload

import uuid
import os
from pprint import pprint
from multiprocessing import Pool
from pathlib import Path
from sadie.airr import Airr, AirrTable
from gspread_pandas import Spread, Client
from sadie.airr import AirrTable
from sadie.airr import constants
from distutils.version import StrictVersion
import math

spread = Spread("Sadie", create_spread=True, create_sheet=True)
pd.set_option("display.max_rows", 45)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 100000)


def fillna(df, fill_value=""):
    """
    Replace null values with `fill_value`.

    Also replaces in categorical columns.
    """
    for col in df.dtypes[df.dtypes == "category"].index:
        if fill_value not in df[col].cat.categories:
            df[col].cat.add_categories([fill_value], inplace=True)
    # Known bug https://github.com/pandas-dev/pandas/issues/25472
    if StrictVersion(pd.__version__) >= StrictVersion("1.0"):
        for col in df.dtypes[
            df.dtypes.apply(lambda x: x in ["float64", "Int16", "Int64"])
        ].index:
            df[col] = df[col].astype("float")
    return df.fillna(fill_value)

## Sadie Annotate

In [87]:
# straight ignore these
ignore = [
    #     "v_call",
    #     "d_call",
    #     "j_call",
    "v_score",
    "d_score",
    "j_score",
    "v_support",
    "d_support",
    "j_support",
]

# cast these to integers
starts_and_ends = [
    "cdr1_end",
    "cdr1_start",
    "cdr2_end",
    "cdr2_start",
    "cdr3_end",
    "cdr3_start",
    "d_alignment_end",
    "d_alignment_start",
    "d_germline_end",
    "d_germline_start",
    "d_sequence_end",
    "d_sequence_start",
    "fwr1_end",
    "fwr1_start",
    "fwr2_end",
    "fwr2_start",
    "fwr4_end",
    "fwr4_start",
    "j_alignment_end",
    "j_alignment_start",
    "j_germline_end",
    "j_germline_start",
    "j_sequence_end",
    "j_sequence_start",
]


def make_sadie_comparable(df):
    if isinstance(df, AirrTable):
        df = df.table
    # comparison keys between imgt and sadie
    compare_key = list(constants.IGBLAST_AIRR.keys()) + [
        "v_call_top",
        "d_call_top",
        "j_call_top",
    ]
    compare_key.remove("v_frameshift")
    df = df[compare_key].drop(ignore, axis=1)
    df.loc[:, starts_and_ends] = df[starts_and_ends].astype("Int64")
    df.insert(
        df.columns.get_loc("v_call_top"),
        "v_gene_top",
        df["v_call_top"].str.split("*").str.get(0),
    )
    df.insert(
        df.columns.get_loc("d_call_top"),
        "d_gene_top",
        df["d_call_top"].str.split("*").str.get(0),
    )
    df.insert(
        df.columns.get_loc("j_call_top"),
        "j_gene_top",
        df["j_call_top"].str.split("*").str.get(0),
    )
    return df


def make_imgt_comparable(df):
    def map_bool(mapp_df, col):
        return mapp_df.loc[:, col].map(
            {"T": True, "F": False, True: True, False: False, nan: False},
            na_action="ignore",
        )

    compare_key = list(constants.IGBLAST_AIRR.keys())
    compare_key.remove("v_frameshift")
    df = df[compare_key].copy()
    bool_cols = ["vj_in_frame", "productive", "rev_comp", "complete_vdj", "stop_codon"]
    for col in bool_cols:
        df[col] = map_bool(df, col)

    upper_columns = [
        "fwr1",
        "cdr1",
        "fwr2",
        "cdr2",
        "fwr3",
        "cdr3",
        "fwr4",
        "germline_alignment",
        "germline_alignment_aa",
        "sequence_alignment",
        "sequence_alignment_aa",
        "v_germline_alignment",
        "v_germline_alignment_aa",
        "v_sequence_alignment",
        "v_sequence_alignment_aa",
        "d_germline_alignment",
        "d_germline_alignment_aa",
        "d_sequence_alignment",
        "d_sequence_alignment_aa",
        "j_germline_alignment",
        "j_germline_alignment_aa",
        "j_sequence_alignment",
        "j_sequence_alignment_aa",
        "np1",
        "np2",
        "junction",
        "sequence",
    ]
    for col in upper_columns:
        df[col] = df[col].str.upper().str.replace(".", "")

    df["v_call_top"] = df["v_call"].str.split(",").str.get(0).str.split().str.get(1)
    df["v_gene_top"] = df["v_call_top"].str.split("*").str.get(0)
    df["d_call_top"] = df["d_call"].str.split(",").str.get(0).str.split("_").str.get(-1)
    df["d_gene_top"] = df["d_call_top"].str.split("*").str.get(0)
    df["j_call_top"] = df["j_call"].str.split(",").str.get(0).str.split().str.get(1)
    df["j_gene_top"] = df["j_call_top"].str.split("*").str.get(0)
    df = df.drop(ignore, axis=1)
    df.loc[:, starts_and_ends] = df[starts_and_ends].astype("Int64")
    return df

In [12]:
sub_sample_file = "../../tests/integration/airr/fixtures/OAS_subsample.fasta"
airr_api = Airr(species="human", database="imgt", functional="all")
sadie_airr = airr_api.run_file(sub_sample_file)

Caution - sequences ['4_704010461_igblastn_anarci_Heavy_IGHE_8', '1327_SRR12190303_igblastn_anarci_Heavy_Bulk_136', '1890_SRR12190307_igblastn_anarci_Heavy_Bulk_469', '5664_SRR11937611_igblastn_anarci_Heavy_Bulk_62', '6073_SRR11610505_1_igblastn_anarci_Bulk_256', '8948_Subject-49_igblastn_anarci_Heavy_IGHE_2', '9858_SRR12190266_igblastn_anarci_Heavy_Bulk_1844', '11672_704010461_igblastn_anarci_Heavy_IGHE_10', '11888_SRR12190262_igblastn_anarci_Heavy_Bulk_654', '19830_G_d9_L_igblastn_anarci_Light_Bulk_791'] may need manual inspections


In [88]:
sadie_comparable = make_sadie_comparable(sadie_airr)

In [89]:
imgt_airr = "../../tests/integration/airr/fixtures/OAS_airr_from_imgtvquest.tsv"
imgt_df = pd.read_csv(imgt_airr, delimiter="\t", low_memory=False)

imgt_comparable = make_imgt_comparable(imgt_df)

<ipython-input-87-ab6def00e12a>:117: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df[col] = df[col].str.upper().str.replace(".", "")


In [164]:
check_these = [
    "sequence",
    "locus",
    "stop_codon",
    "vj_in_frame",
    "productive",
    "rev_comp",
    "complete_vdj",
    "v_call",
    "v_call_top",
    "v_gene_top",
    "d_call_top",
    "d_gene_top",
    "j_call_top",
    "j_gene_top",
    "fwr1",
    "cdr1",
    "fwr2",
    "cdr2",
    "fwr3",
    "cdr3",
    "fwr4",
    "fwr1_aa",
    "cdr1_aa",
    "fwr2_aa",
    "cdr2_aa",
    "fwr3_aa",
    "cdr3_aa",
    "fwr4_aa",
]

In [165]:
compare = imgt_comparable[check_these] == sadie_comparable[check_these]

In [166]:
compare.describe()

,sequence,locus,stop_codon,vj_in_frame,productive,rev_comp,complete_vdj,v_call,v_call_top,v_gene_top,d_call_top,d_gene_top,j_call_top,j_gene_top,fwr1,cdr1,fwr2,cdr2,fwr3,cdr3,fwr4,fwr1_aa,cdr1_aa,fwr2_aa,cdr2_aa,fwr3_aa,cdr3_aa,fwr4_aa
count,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000
unique,1,1,2,2,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,True,True,True,False,True,True,False,False,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True
freq,25000,25000,13650,24808,24718,25000,24855,25000,23742,24535,15027,14931,23810,23969,24253,24383,24577,24745,24861,24767,17484,24256,24388,24581,24751,24825,24720,23893


In [167]:
# you cannot trust fwr4 since it has the extra nucleotide in the jcall

In [168]:
from IPython.display import clear_output
from IPython.core.display import display, HTML

In [172]:
pd.set_option("display.max_rows", 100)
for i in range(6,20):
    single_entry_diff = (
        imgt_comparable.loc[compare[~compare["v_gene_top"]].index]
        .iloc[i]
        .to_frame()
        .join(
            sadie_comparable.loc[compare[~compare["v_gene_top"]].index].iloc[i].to_frame(),
            rsuffix="_sadie",
            lsuffix="_imgt",
        )
        .join(compare[~compare["v_gene_top"]].iloc[i].to_frame())
        .fillna("")
    ).loc[check_these]
    print(imgt_comparable.loc[compare[~compare["v_gene_top"]].index].iloc[i]['sequence'])
    display(HTML(single_entry_diff._repr_html_()))
    input()
    clear_output()

CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTCAGTTATTTTGGCATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACTGGAGTGGGTGGCACTGATAAAGTATGATGGAAGTAATACATACCACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACACTATATTTGCAAATGAACAGCCTGAGACCCGAGGACACGGCCATATATTACTGTGCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCGGCTGCTTTGACTCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCGCCCTGCTCCAGGAGCACCTCTGGGGGCACAGCGGCCCTGGGCTGCCTG


,431_imgt,431_sadie,431
sequence,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,True
locus,IGH,IGH,True
stop_codon,True,False,False
vj_in_frame,True,True,True
productive,True,True,True
rev_comp,False,False,True
complete_vdj,False,False,True
v_call,"Homsap IGHV3-30*02 F, or Homsap IGHV3-30-5*02 ...",IGHV3-33*06,False
v_call_top,IGHV3-30*02,IGHV3-33*06,False
v_gene_top,IGHV3-30,IGHV3-33,False


KeyboardInterrupt: Interrupted by user

In [175]:
airr_api.run_single(
    "test",
    "CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTCAGTTATTTTGGCATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACTGGAGTGGGTGGCACTGATAAAGTATGATGGAAGTAATACATACCACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACACTATATTTGCAAATGAACAGCCTGAGACCCGAGGACACGGCCATATATTACTGTGCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCGGCTGCTTTGACTCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCGCCCTGCTCCAGGAGCACCTCTGGGGGCACAGCGGCCCTGGGCTGCCTG",
)

,sequence_id,sequence,species,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call_top,v_call,d_call_top,d_call,j_call_top,j_call,sequence_alignment,germline_alignment,sequence_alignment_aa,germline_alignment_aa,v_alignment_start,v_alignment_end,d_alignment_start,d_alignment_end,j_alignment_start,j_alignment_end,v_sequence_alignment,v_sequence_alignment_aa,v_germline_alignment,v_germline_alignment_aa,d_sequence_alignment,d_sequence_alignment_aa,d_germline_alignment,d_germline_alignment_aa,j_sequence_alignment,j_sequence_alignment_aa,j_germline_alignment,j_germline_alignment_aa,fwr1,fwr1_aa,cdr1,cdr1_aa,fwr2,fwr2_aa,cdr2,cdr2_aa,fwr3,fwr3_aa,fwr4,fwr4_aa,cdr3,cdr3_aa,junction,junction_length,junction_aa,junction_aa_length,v_score,d_score,j_score,v_cigar,d_cigar,j_cigar,v_support,d_support,j_support,v_identity,d_identity,j_identity,v_sequence_start,v_sequence_end,v_germline_start,v_germline_end,d_sequence_start,d_sequence_end,d_germline_start,d_germline_end,j_sequence_start,j_sequence_end,j_germline_start,j_germline_end,fwr1_start,fwr1_end,cdr1_start,cdr1_end,fwr2_start,fwr2_end,cdr2_start,cdr2_end,fwr3_start,fwr3_end,fwr4_start,fwr4_end,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,vdj_nt,vdj_aa
0,test,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,human,IGH,False,True,False,True,False,False,IGHV3-33*06,IGHV3-33*06,IGHD5-24*01,IGHD5-24*01,IGHJ4*02,IGHJ4*02,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,TGGGAGGTCCCTGAGACTCTCCTGTGCAGCGTCTGGATTCACCTTC...,GESLKISCAASGFTFSYFGMHWVRQAPGKGLEWVALIKYDGSNTYH...,GRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVIWYDGSNKYY...,1,255,269,279,292,338,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,GESLKISCAASGFTFSYFGMHWVRQAPGKGLEWVALIKYDGSNTYH...,TGGGAGGTCCCTGAGACTCTCCTGTGCAGCGTCTGGATTCACCTTC...,GRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVIWYDGSNKYY...,TGGCTACAATT,WLQ,TGGCTACAATT,WLQ,CTGCTTTGACTCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,CFDSWGQGTLVTVSS,CTACTTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,YFDYWGQGTLVTVSS,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCT,GESLKISCAAS,GGATTCACGTTCAGTTATTTTGGC,GFTFSYFG,ATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACTGGAGTGGGTGG...,MHWVRQAPGKGLEWVAL,ATAAAGTATGATGGAAGTAATACA,IKYDGSNT,TACCACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACA...,YHADSVKGRFTISRDNSKNTLYLQMNSLRPEDTAIYYC,TGGGGCCAGGGAACCCTGGTCACCGTCTCCTCA,WGQGTLVTVSS,GCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCGGCT...,AKEMDMRWLQFHDFGCFDS,TGTGCGAAAGAGATGGATATGCGGTGGCTACAATTTCACGATTTCG...,63,CAKEMDMRWLQFHDFGCFDSW,21,343.0,21.84375,69.75,41N255M166S,268S7N11M142S2N,291S1N47M83S,4.067000e-96,0.06912,3.339000e-16,89.4375,100.0,95.75,1,255,42,296,269,279,8.0,18.0,292,338,2,48,1,34,35,58,59,109,110,133,134,247,305,337,248,304,GATGGATATGCGG,13,TCACGATTTCGG,12,CGGGGAGTCTCTGAAGATCTCCTGTGCAGCGTCTGGATTCACGTTC...,GESLKISCAASGFTFSYFGMHWVRQAPGKGLEWVALIKYDGSNTYH...


In [176]:
airr_api

IgBLAST: env IGDATA=/Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/bin/darwin/igblastn -min_D_match 5 -num_alignments_V 5 -num_alignments_J 3 -num_alignments_D 3 -organism human -ig_seqtype Ig -germline_db_V /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig/blastdb/human_V -germline_db_D /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig/blastdb/human_D -germline_db_J /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/all/Ig/blastdb/human_J -auxiliary_data /Users/jordanwillis/repos/personal/sadie/src/sadie/airr/data/germlines/imgt/aux_db/human_gl.aux -outfmt 19 -domain_system imgt -word_size 11 -gapopen 5 -gapextend 2 -J_penalty -1 -V_penalty -2 -num_threads 12 -show_translation -extend_align5end -extend_align3end

In [112]:
imgt_comparable.loc[55, "sequence"]

'CTTCGGAGACCCTGTCCCTCACCTGCGCTGTCTATGGTGGGTCCTTCAGTGGTTACTACTGGGGCTGGATCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATTATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCCGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCTGTGTATTACTGTGCGAACGGCCCTATGTATAGTGGGAGCTACGGGTGGTTCGACCCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGTAAG'

## Read OAS

In [5]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample.bz2",
    index_col=0,
).reset_index()

sub_sample_file = "../../tests/integration/airr/fixtures/OAS_subsample.fasta"
with open("../../tests/integration/airr/fixtures/OAS_subsample.fasta", "w") as f:
    for index, (i, k, j) in enumerate(
        zip(
            df_bz2["OAS_dataunit"].str.split(".csv.gz").str.get(0),
            df_bz2["OAS_dataindex"],
            df_bz2["sequence"],
        )
    ):
        f.write(f">{index}_{i}_{k}\n{j}\n")

## IMGT HI-Vqeust "AIRR" format

In [17]:
spread.df_to_sheet(
    imgt_df.head(100).set_index("sequence_id"),
    sheet="imgt",
    replace=True,
    freeze_headers=True,
    freeze_index=True,
)

In [ ]:
# spreadable_df = fillna(sadie_airr.table.head(100))
spread.df_to_sheet(
    spreadable_df.set_index("sequence_id"),
    sheet="sadie",
    replace=True,
    freeze_headers=True,
    freeze_index=True,
)

## Clean UP IMGT

## Clean up sadie

In [358]:
ref[check_these] == target[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2    False
fwr3     True
cdr3     True
Name: 99, dtype: bool

In [362]:
ref["cdr2_aa"]

'ISGSRTYI'

In [363]:
target["cdr2_aa"]

'ISGSFRTYI'

In [361]:
ref["sequence"]

'TACCTTGACTTAACTCTTGGGGGTCTGGGGGAGGCCTGGTCAAGCCTGGGGGGTCCCTGAGACTCTCATGTGCAGCCTCTGGATTCACCTTCAGTAGTTTTACCATGAATTGGGTCCGCCAGGCTCCAGGGAAGGGACTGGAGTGGGTCTCATCCATTAGCGGCAGTTTCAGAACTTACATATATTATGCCGACTCAGTGAAGGGCCGATTCACCGTCTCCAGAGACAACGCCAAGGAATTGGTGTTTCTGCAGATGGACAACCTGAGAGTCGAAGACACAGGTGTATATTATTGTGCGAGAGACCTAAATACGGTGACTACCCCAGAATACTTCCAACACTGGGGCCTGGGCACCCCGGTCTCCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTG'

In [355]:
target["cdr2"]

'ATCAACCCTAGTGGTGATAACACA'

In [344]:
ref["fwr4"]

'TGGGGCCAGGGAACC'

In [345]:

target["fwr4"]

'TGGGGCCAGGGAACCCT'

In [342]:
ref[check_these] == target[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2     True
fwr3     True
cdr3     True
fwr4    False
Name: 3, dtype: bool

In [328]:
len(ref["fwr4"])

33

In [320]:
ref["v_call_top"]

'IGHV4-31*02'

In [321]:
target["v_call_top"]

'IGHV4-31*02'

In [303]:
ref["j_call_top"]

'IGHJ4*02'

In [304]:
target["j_call_top"]

'IGHJ4*02'

In [330]:
len(target["fwr4"])

33

In [331]:
len(ref["fwr4"])

33

In [ ]:
TGGGGCCAGGGAACCCTGGTCACCGTCGCCTCAG
TGGGGCCAGGGAACCCTGGTCACCGTCGCCTCA

In [312]:
from Bio.Seq import Seq

Seq("tggggccagggaaccctggtcaccgtcgcctca").translate()

Seq('WGQGTLVTVAS')

In [308]:
ref["sequence"]

'GGCCTCAGTGAAGGTCTCCTGCAAGGCATCTGGATACACCTTCACCAACCACTACATCCACTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGAATAATCAACCCTAGTGGTGGTAGGACAAGTCACGTACAGGAGTTCCAGGGCAGAGTCACCATGACCAGGGACACGTCCACGAGTACTGTCTACATGGAGTTGAGTAGCCTGAGATCTGAGGACACGGCCGTGTACTTTTGTGCGAGACAACAGTGTAGTGCTTCCAACTGCTATGAGGAGAGTTTTGACCACTGGGGCCAGGGAACCCTGGTCACCGTCGCCTCAGCCTCCACACAGAGCCCATCCGTCTTC'

In [298]:
target[check_these] == ref[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2     True
fwr3     True
cdr3     True
fwr4    False
Name: 1_705010661_igblastn_anarci_Heavy_IGHE_1125, dtype: bool

'ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCTGGATCCGCCAGCACCCAGGGAAGGGCCTGGAGTGGATTGGGTACATCTATTACAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTTACCATATCAGTAGACACGTCTAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACTGCCGCGGACACGGCCGTGTATTACTGTGCGANNNNNNNNNAACTCCNNNNTGATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCA'

In [193]:
spread.df_to_sheet(
    fillna(top_100_sadie_airr_only),
    sheet="sadie_airr_only",
    freeze_headers=True,
    freeze_index=True,
    replace=True,
)

<ipython-input-192-d3342252a2a0>:16: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  df.dtypes.apply(lambda x: x in ["float64", "Int16"])


In [213]:
spread.df_to_sheet(
    fillna(top_100_imgt_airr_only),
    sheet="imgt_airr_only",
    freeze_headers=True,
    freeze_index=True,
    replace=True,
)

<ipython-input-210-72c7d47bbfa2>:16: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  df.dtypes.apply(lambda x: x in ["float64", "Int16", "Int64"])


germline_alignment         ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCT...
germline_alignment_aa      TVSGGSISSGGYYWSWIRQHPGKGLEWIGYIYYSGSTYYNPSLKSR...
v_alignment_end                                                       316.00
v_germline_start                                                        1.00
v_germline_end                                                        316.00
v_identity                                                             82.22
d_alignment_start                                                     325.00
d_alignment_end                                                       334.00
d_germline_alignment                                              CAACTGGAAC
d_germline_alignment_aa                                                  QLE
d_germline_start                                                        5.00
d_germline_end                                                         14.00
d_identity                                                              0.00

## Try and find OAS similariities

I'm suspect of the entire airr file since it doesn't have fwr4

In [ ]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample.bz2",
    index_col=0,
).reset_index()

In [ ]:
airr_dataframe = airr_api.run_dataframe(df_bz2, "index", "sequence")

In [ ]:
ref = df_bz2[df_bz2.columns.intersection(airr_dataframe.table.columns)].drop(
    "sequence_id", axis=1
)
target = airr_dataframe.table[
    airr_dataframe.table.columns.intersection(df_bz2.columns)
].drop("sequence_id", axis=1)

In [ ]:
ref.re

In [ ]:
for index in ref.index:
    sub_ref = ref.loc[index]
    sub_target = target.loc[index]
    aggree = sub_ref[sub_ref == sub_target].index
    disagree = sub_ref[sub_ref != sub_target].index
    if disagree.empty:
        print('yay')
    for dis_index in disagree:
        print(f"OAR-{dis_index}:{sub_ref[dis_index]}")
        print(f"Sadie-{dis_index}:{sub_target[dis_index]}\n")

    break

In [ ]:
disagree

In [ ]:
from pandas.testing import assert_series_equal

# Get IMGT airr

In [ ]:
imgt_airr = "../../tests/integration/airr/fixtures/OAS_airr_from_imgtvquest.tsv"
imgt_df = pd.read_csv(imgt_airr, delimiter="\t")

In [ ]:
spread.df_to_sheet(imgt_df, sheet="imgt_airr")